<a href="https://colab.research.google.com/github/Grivine-19/Coronavirus-Sentiment-Analysis-Kenya/blob/master/Coronavirus_EDA_Kenya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**Steps**</center>



*   Tokenization
*   Cleaning the data
*   Removing Stop words
*   Classification task;

  * *Positive, negative,neutral*
*   Apply supervised algorithm for classification;

  * *Bag of Words*
  * *Lexicons-pre classified set of words*

*   Calculation - *final sentiment score*







In [ ]:
pip install pyLDAvis

In [ ]:
pip install fasttext

In [ ]:
pip install contractions

# <center>**Exploratory Data Analysis**</center>

*Here, we clean and visualize data from Data_Extraction.ipynb file to gain insights on the kind of data we are dealing with.*

In [ ]:
#Import Visualization Libraries

from google.colab import drive
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import numpy as np
import re
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pickle
import pyLDAvis.sklearn
from collections import Counter
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

#Import data cleaning Libraries

from html.parser import HTMLParser

import string
import fasttext
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

#Default Parameters

%matplotlib inline
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)

In [ ]:
#Mount the google drive folder

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# <center>**Nairobi; Data Cleaning**</center>

In [ ]:
#Load the dataframe

nairobi_tweets = pd.read_csv('/content/gdrive/My Drive/Tweets/nairobi.csv')

nairobi_tweets.head()

,Unnamed: 0,ID,DATE,TWEET,LOCATION,RETWEETS,FAVORITES,HASHTAGS
0,0,1289314486092877825,2020-07-31 21:38:22+00:00,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,NaN,0,1,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy
1,1,1215096387550896128,2020-01-09 02:22:08+00:00,"Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...",NaN,28,108,NaN
2,2,1289257452219793408,2020-07-31 17:51:44+00:00,#COVID19KE #TheScoreKE #ruiru,NaN,0,2,#COVID19KE #TheScoreKE #ruiru
3,3,1289191573054345219,2020-07-31 13:29:57+00:00,In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,NaN,10,28,#BuildBackBetter #SDGs #COVID19KE
4,4,1289142305593724928,2020-07-31 10:14:11+00:00,Wishing him and all front line workers quick recovery from #COVID19KE.,NaN,0,0,#COVID19KE


In [ ]:
nairobi_tweets.info

<bound method DataFrame.info of       Unnamed: 0  ...                                                                                             HASHTAGS
0              0  ...             #FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy
1              1  ...                                                                                                  NaN
2              2  ...                                                                        #COVID19KE #TheScoreKE #ruiru
3              3  ...                                                                    #BuildBackBetter #SDGs #COVID19KE
4              4  ...                                                                                           #COVID19KE
...          ...  ...                                                                                                  ...
8636         225  ...                                                                                   #lo

In [ ]:
#List the column names

for col in nairobi_tweets:
  print (col)

Unnamed: 0
ID
DATE
TWEET
LOCATION
RETWEETS
FAVORITES
HASHTAGS


In [ ]:
#Drop the 'location' and 'Unnamed 0:' columns from the df

nairobi_tweets.drop(["Unnamed: 0", "LOCATION"], axis = 1, inplace = True) 

In [ ]:
# Save the updated dataframe to a new csv file

nairobi_tweets.to_csv('/content/gdrive/My Drive/Tweets/nairobi_final.csv')

In [ ]:
#Load the updated csv file

new_df = pd.read_csv('/content/gdrive/My Drive/Tweets/nairobi_final.csv')

#get the shape of the dataframe
print(new_df.shape)

new_df.head()  #the Unnamed: 0 column does not delete, it resulted from the merged datasets of tweets fetched from Nairobi.

(8641, 7)


,Unnamed: 0,ID,DATE,TWEET,RETWEETS,FAVORITES,HASHTAGS
0,0,1289314486092877825,2020-07-31 21:38:22+00:00,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,0,1,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy
1,1,1215096387550896128,2020-01-09 02:22:08+00:00,"Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...",28,108,NaN
2,2,1289257452219793408,2020-07-31 17:51:44+00:00,#COVID19KE #TheScoreKE #ruiru,0,2,#COVID19KE #TheScoreKE #ruiru
3,3,1289191573054345219,2020-07-31 13:29:57+00:00,In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,10,28,#BuildBackBetter #SDGs #COVID19KE
4,4,1289142305593724928,2020-07-31 10:14:11+00:00,Wishing him and all front line workers quick recovery from #COVID19KE.,0,0,#COVID19KE


In [ ]:
#Check for the max and min dates in the data frame
#not correct as such in some instances

print(f" Data Available since {new_df.DATE.min()}")
print(f" Data Available upto {new_df.DATE.max()}")

 Data Available since 2019-11-09 11:23:40+00:00
 Data Available upto 2020-08-14 13:13:25+00:00


In [ ]:
#Checking for null values

new_df.isnull().mean()*100 

print(new_df.isnull().sum())

'''There are null values in the hashtags column...
Droping these 1159 NAN values in the hashtag column would result in the loss of similar rows of data.
Missing values causes problems with predictor models such as SVM, we retain the NAN's because our
problem requires a classification model.'''

#Do not run lines 13 & 14 of the cell!!
'''new_df.dropna(inplace=True)
print(new_df.shape)'''

Unnamed: 0       0
ID               0
DATE             0
TWEET            0
RETWEETS         0
FAVORITES        0
HASHTAGS      1159
dtype: int64


'new_df.dropna(inplace=True)\nprint(new_df.shape)'

In [ ]:
#Removing contractions from the tweet column

new_df['UNCONTRACTED'] = new_df['TWEET'].apply(lambda x: [contractions.fix(word) for word in x.split()])
new_df.head()

,Unnamed: 0,ID,DATE,TWEET,RETWEETS,FAVORITES,HASHTAGS,UNCONTRACTED
0,0,1289314486092877825,2020-07-31 21:38:22+00:00,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,0,1,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy,"[#FACupFinal, #ruiru, #lockdownextension, #COVID19KE, #TheUmbrellaAcademy, https://m.facebook.co..."
1,1,1215096387550896128,2020-01-09 02:22:08+00:00,"Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...",28,108,NaN,"[Check, out, Audio, Book, Elven, Magic, on, Audible,, amazon, pay, $75, for, every, bounty(have,..."
2,2,1289257452219793408,2020-07-31 17:51:44+00:00,#COVID19KE #TheScoreKE #ruiru,0,2,#COVID19KE #TheScoreKE #ruiru,"[#COVID19KE, #TheScoreKE, #ruiru]"
3,3,1289191573054345219,2020-07-31 13:29:57+00:00,In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,10,28,#BuildBackBetter #SDGs #COVID19KE,"[In, the, opening, remarks, @Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, us..."
4,4,1289142305593724928,2020-07-31 10:14:11+00:00,Wishing him and all front line workers quick recovery from #COVID19KE.,0,0,#COVID19KE,"[Wishing, him, and, all, front, line, workers, quick, recovery, from, #COVID19KE.]"


In [ ]:
#Converting the Uncontracted twwets to a string

new_df['UNCONTRACTED_STR'] = [' '.join(map(str, l)) for l in new_df['UNCONTRACTED']]
new_df.head()

,Unnamed: 0,ID,DATE,TWEET,RETWEETS,FAVORITES,HASHTAGS,UNCONTRACTED,UNCONTRACTED_STR
0,0,1289314486092877825,2020-07-31 21:38:22+00:00,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,0,1,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy,"[#FACupFinal, #ruiru, #lockdownextension, #COVID19KE, #TheUmbrellaAcademy, https://m.facebook.co...",#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...
1,1,1215096387550896128,2020-01-09 02:22:08+00:00,"Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...",28,108,NaN,"[Check, out, Audio, Book, Elven, Magic, on, Audible,, amazon, pay, $75, for, every, bounty(have,...","Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member..."
2,2,1289257452219793408,2020-07-31 17:51:44+00:00,#COVID19KE #TheScoreKE #ruiru,0,2,#COVID19KE #TheScoreKE #ruiru,"[#COVID19KE, #TheScoreKE, #ruiru]",#COVID19KE #TheScoreKE #ruiru
3,3,1289191573054345219,2020-07-31 13:29:57+00:00,In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,10,28,#BuildBackBetter #SDGs #COVID19KE,"[In, the, opening, remarks, @Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, us...",In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...
4,4,1289142305593724928,2020-07-31 10:14:11+00:00,Wishing him and all front line workers quick recovery from #COVID19KE.,0,0,#COVID19KE,"[Wishing, him, and, all, front, line, workers, quick, recovery, from, #COVID19KE.]",Wishing him and all front line workers quick recovery from #COVID19KE.


In [ ]:
#English Language Identification

pretrained_model = "lid.176.bin" 
model = fasttext.load_model(pretrained_model)langs=[]
for sent in new_df['UNCONTRACTED_STR']:
    lang = model.predict(sent)[0]
    langs.append(str(lang)[11:13])
new_df['langs'] = langs

SyntaxError: ignored

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Applying Tokenizer Function

new_df['TOKENIZED'] = new_df['UNCONTRACTED_STR'].apply(word_tokenize)

new_df.head()

,Unnamed: 0,ID,DATE,TWEET,RETWEETS,FAVORITES,HASHTAGS,UNCONTRACTED,UNCONTRACTED_STR,TOKENIZED
0,0,1289314486092877825,2020-07-31 21:38:22+00:00,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,0,1,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy,"[#FACupFinal, #ruiru, #lockdownextension, #COVID19KE, #TheUmbrellaAcademy, https://m.facebook.co...",#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,"[#, FACupFinal, #, ruiru, #, lockdownextension, #, COVID19KE, #, TheUmbrellaAcademy, https, :, /..."
1,1,1215096387550896128,2020-01-09 02:22:08+00:00,"Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...",28,108,NaN,"[Check, out, Audio, Book, Elven, Magic, on, Audible,, amazon, pay, $75, for, every, bounty(have,...","Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...","[Check, out, Audio, Book, Elven, Magic, on, Audible, ,, amazon, pay, $, 75, for, every, bounty, ..."
2,2,1289257452219793408,2020-07-31 17:51:44+00:00,#COVID19KE #TheScoreKE #ruiru,0,2,#COVID19KE #TheScoreKE #ruiru,"[#COVID19KE, #TheScoreKE, #ruiru]",#COVID19KE #TheScoreKE #ruiru,"[#, COVID19KE, #, TheScoreKE, #, ruiru]"
3,3,1289191573054345219,2020-07-31 13:29:57+00:00,In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,10,28,#BuildBackBetter #SDGs #COVID19KE,"[In, the, opening, remarks, @Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, us...",In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,"[In, the, opening, remarks, @, Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, ..."
4,4,1289142305593724928,2020-07-31 10:14:11+00:00,Wishing him and all front line workers quick recovery from #COVID19KE.,0,0,#COVID19KE,"[Wishing, him, and, all, front, line, workers, quick, recovery, from, #COVID19KE.]",Wishing him and all front line workers quick recovery from #COVID19KE.,"[Wishing, him, and, all, front, line, workers, quick, recovery, from, #, COVID19KE, .]"


In [ ]:
#Converting all characters to lowercase

new_df['LOWER'] = new_df['TOKENIZED'].apply(lambda x: [word.lower() for word in x])

new_df.head()

,Unnamed: 0,ID,DATE,TWEET,RETWEETS,FAVORITES,HASHTAGS,UNCONTRACTED,UNCONTRACTED_STR,TOKENIZED,LOWER
0,0,1289314486092877825,2020-07-31 21:38:22+00:00,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,0,1,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy,"[#FACupFinal, #ruiru, #lockdownextension, #COVID19KE, #TheUmbrellaAcademy, https://m.facebook.co...",#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,"[#, FACupFinal, #, ruiru, #, lockdownextension, #, COVID19KE, #, TheUmbrellaAcademy, https, :, /...","[#, facupfinal, #, ruiru, #, lockdownextension, #, covid19ke, #, theumbrellaacademy, https, :, /..."
1,1,1215096387550896128,2020-01-09 02:22:08+00:00,"Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...",28,108,NaN,"[Check, out, Audio, Book, Elven, Magic, on, Audible,, amazon, pay, $75, for, every, bounty(have,...","Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...","[Check, out, Audio, Book, Elven, Magic, on, Audible, ,, amazon, pay, $, 75, for, every, bounty, ...","[check, out, audio, book, elven, magic, on, audible, ,, amazon, pay, $, 75, for, every, bounty, ..."
2,2,1289257452219793408,2020-07-31 17:51:44+00:00,#COVID19KE #TheScoreKE #ruiru,0,2,#COVID19KE #TheScoreKE #ruiru,"[#COVID19KE, #TheScoreKE, #ruiru]",#COVID19KE #TheScoreKE #ruiru,"[#, COVID19KE, #, TheScoreKE, #, ruiru]","[#, covid19ke, #, thescoreke, #, ruiru]"
3,3,1289191573054345219,2020-07-31 13:29:57+00:00,In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,10,28,#BuildBackBetter #SDGs #COVID19KE,"[In, the, opening, remarks, @Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, us...",In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,"[In, the, opening, remarks, @, Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, ...","[in, the, opening, remarks, @, ezra_k_yego, mentioned, that, it, is, extremely, important, for, ..."
4,4,1289142305593724928,2020-07-31 10:14:11+00:00,Wishing him and all front line workers quick recovery from #COVID19KE.,0,0,#COVID19KE,"[Wishing, him, and, all, front, line, workers, quick, recovery, from, #COVID19KE.]",Wishing him and all front line workers quick recovery from #COVID19KE.,"[Wishing, him, and, all, front, line, workers, quick, recovery, from, #, COVID19KE, .]","[wishing, him, and, all, front, line, workers, quick, recovery, from, #, covid19ke, .]"


In [ ]:
#Removing punctuations

punc = string.punctuation
new_df['NO_PUNC'] = new_df['LOWER'].apply(lambda x: [word for word in x if word not in punc])
new_df.head()

,Unnamed: 0,ID,DATE,TWEET,RETWEETS,FAVORITES,HASHTAGS,UNCONTRACTED,UNCONTRACTED_STR,TOKENIZED,LOWER,NO_PUNC
0,0,1289314486092877825,2020-07-31 21:38:22+00:00,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,0,1,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy,"[#FACupFinal, #ruiru, #lockdownextension, #COVID19KE, #TheUmbrellaAcademy, https://m.facebook.co...",#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,"[#, FACupFinal, #, ruiru, #, lockdownextension, #, COVID19KE, #, TheUmbrellaAcademy, https, :, /...","[#, facupfinal, #, ruiru, #, lockdownextension, #, covid19ke, #, theumbrellaacademy, https, :, /...","[facupfinal, ruiru, lockdownextension, covid19ke, theumbrellaacademy, https, //m.facebook.com/st..."
1,1,1215096387550896128,2020-01-09 02:22:08+00:00,"Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...",28,108,NaN,"[Check, out, Audio, Book, Elven, Magic, on, Audible,, amazon, pay, $75, for, every, bounty(have,...","Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...","[Check, out, Audio, Book, Elven, Magic, on, Audible, ,, amazon, pay, $, 75, for, every, bounty, ...","[check, out, audio, book, elven, magic, on, audible, ,, amazon, pay, $, 75, for, every, bounty, ...","[check, out, audio, book, elven, magic, on, audible, amazon, pay, 75, for, every, bounty, have, ..."
2,2,1289257452219793408,2020-07-31 17:51:44+00:00,#COVID19KE #TheScoreKE #ruiru,0,2,#COVID19KE #TheScoreKE #ruiru,"[#COVID19KE, #TheScoreKE, #ruiru]",#COVID19KE #TheScoreKE #ruiru,"[#, COVID19KE, #, TheScoreKE, #, ruiru]","[#, covid19ke, #, thescoreke, #, ruiru]","[covid19ke, thescoreke, ruiru]"
3,3,1289191573054345219,2020-07-31 13:29:57+00:00,In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,10,28,#BuildBackBetter #SDGs #COVID19KE,"[In, the, opening, remarks, @Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, us...",In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,"[In, the, opening, remarks, @, Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, ...","[in, the, opening, remarks, @, ezra_k_yego, mentioned, that, it, is, extremely, important, for, ...","[in, the, opening, remarks, ezra_k_yego, mentioned, that, it, is, extremely, important, for, us,..."
4,4,1289142305593724928,2020-07-31 10:14:11+00:00,Wishing him and all front line workers quick recovery from #COVID19KE.,0,0,#COVID19KE,"[Wishing, him, and, all, front, line, workers, quick, recovery, from, #COVID19KE.]",Wishing him and all front line workers quick recovery from #COVID19KE.,"[Wishing, him, and, all, front, line, workers, quick, recovery, from, #, COVID19KE, .]","[wishing, him, and, all, front, line, workers, quick, recovery, from, #, covid19ke, .]","[wishing, him, and, all, front, line, workers, quick, recovery, from, covid19ke]"


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Removing stopwords

stop_words = set(stopwords.words('english'))
new_df['STOPWORDS_REMOVED'] = new_df['NO_PUNC'].apply(lambda x: [word for word in x if word not in stop_words])
new_df.head()

,Unnamed: 0,ID,DATE,TWEET,RETWEETS,FAVORITES,HASHTAGS,UNCONTRACTED,UNCONTRACTED_STR,TOKENIZED,LOWER,NO_PUNC,STOPWORDS_REMOVED
0,0,1289314486092877825,2020-07-31 21:38:22+00:00,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,0,1,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy,"[#FACupFinal, #ruiru, #lockdownextension, #COVID19KE, #TheUmbrellaAcademy, https://m.facebook.co...",#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,"[#, FACupFinal, #, ruiru, #, lockdownextension, #, COVID19KE, #, TheUmbrellaAcademy, https, :, /...","[#, facupfinal, #, ruiru, #, lockdownextension, #, covid19ke, #, theumbrellaacademy, https, :, /...","[facupfinal, ruiru, lockdownextension, covid19ke, theumbrellaacademy, https, //m.facebook.com/st...","[facupfinal, ruiru, lockdownextension, covid19ke, theumbrellaacademy, https, //m.facebook.com/st..."
1,1,1215096387550896128,2020-01-09 02:22:08+00:00,"Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...",28,108,NaN,"[Check, out, Audio, Book, Elven, Magic, on, Audible,, amazon, pay, $75, for, every, bounty(have,...","Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...","[Check, out, Audio, Book, Elven, Magic, on, Audible, ,, amazon, pay, $, 75, for, every, bounty, ...","[check, out, audio, book, elven, magic, on, audible, ,, amazon, pay, $, 75, for, every, bounty, ...","[check, out, audio, book, elven, magic, on, audible, amazon, pay, 75, for, every, bounty, have, ...","[check, audio, book, elven, magic, audible, amazon, pay, 75, every, bounty, member, 91, days, co..."
2,2,1289257452219793408,2020-07-31 17:51:44+00:00,#COVID19KE #TheScoreKE #ruiru,0,2,#COVID19KE #TheScoreKE #ruiru,"[#COVID19KE, #TheScoreKE, #ruiru]",#COVID19KE #TheScoreKE #ruiru,"[#, COVID19KE, #, TheScoreKE, #, ruiru]","[#, covid19ke, #, thescoreke, #, ruiru]","[covid19ke, thescoreke, ruiru]","[covid19ke, thescoreke, ruiru]"
3,3,1289191573054345219,2020-07-31 13:29:57+00:00,In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,10,28,#BuildBackBetter #SDGs #COVID19KE,"[In, the, opening, remarks, @Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, us...",In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,"[In, the, opening, remarks, @, Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, ...","[in, the, opening, remarks, @, ezra_k_yego, mentioned, that, it, is, extremely, important, for, ...","[in, the, opening, remarks, ezra_k_yego, mentioned, that, it, is, extremely, important, for, us,...","[opening, remarks, ezra_k_yego, mentioned, extremely, important, us, come, inclusive, collaborat..."
4,4,1289142305593724928,2020-07-31 10:14:11+00:00,Wishing him and all front line workers quick recovery from #COVID19KE.,0,0,#COVID19KE,"[Wishing, him, and, all, front, line, workers, quick, recovery, from, #COVID19KE.]",Wishing him and all front line workers quick recovery from #COVID19KE.,"[Wishing, him, and, all, front, line, workers, quick, recovery, from, #, COVID19KE, .]","[wishing, him, and, all, front, line, workers, quick, recovery, from, #, covid19ke, .]","[wishing, him, and, all, front, line, workers, quick, recovery, from, covid19ke]","[wishing, front, line, workers, quick, recovery, covid19ke]"


In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#Stemming VS Lematization

'''Stemming is certainly the less complicated method but it often does not produce a dictionary-specific morphological root 
of the word. let’s apply lemmatization to our data but it requires some additional steps compared to stemming.'''

#Apply parts of speech tags

new_df['POS_TAGS'] = new_df['STOPWORDS_REMOVED'].apply(nltk.tag.pos_tag)

#Convert POS tags to Wordnet format
'''write a function which make the proper conversion and then use the function within 
a list comprehension to apply the conversion.'''

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

new_df['WORDNET_POS'] = new_df['POS_TAGS'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
#new_df.head()

#Apply NLTK's word Lemmatizer

wnl = WordNetLemmatizer()
new_df['LEMMATIZED'] = new_df['WORDNET_POS'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
new_df.head()

,Unnamed: 0,ID,DATE,TWEET,RETWEETS,FAVORITES,HASHTAGS,UNCONTRACTED,UNCONTRACTED_STR,TOKENIZED,LOWER,NO_PUNC,STOPWORDS_REMOVED,POS_TAGS,WORDNET_POS,LEMMATIZED
0,0,1289314486092877825,2020-07-31 21:38:22+00:00,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,0,1,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy,"[#FACupFinal, #ruiru, #lockdownextension, #COVID19KE, #TheUmbrellaAcademy, https://m.facebook.co...",#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,"[#, FACupFinal, #, ruiru, #, lockdownextension, #, COVID19KE, #, TheUmbrellaAcademy, https, :, /...","[#, facupfinal, #, ruiru, #, lockdownextension, #, covid19ke, #, theumbrellaacademy, https, :, /...","[facupfinal, ruiru, lockdownextension, covid19ke, theumbrellaacademy, https, //m.facebook.com/st...","[facupfinal, ruiru, lockdownextension, covid19ke, theumbrellaacademy, https, //m.facebook.com/st...","[(facupfinal, JJ), (ruiru, NN), (lockdownextension, NN), (covid19ke, NN), (theumbrellaacademy, N...","[(facupfinal, a), (ruiru, n), (lockdownextension, n), (covid19ke, n), (theumbrellaacademy, n), (...","[facupfinal, ruiru, lockdownextension, covid19ke, theumbrellaacademy, http, //m.facebook.com/sto..."
1,1,1215096387550896128,2020-01-09 02:22:08+00:00,"Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...",28,108,NaN,"[Check, out, Audio, Book, Elven, Magic, on, Audible,, amazon, pay, $75, for, every, bounty(have,...","Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...","[Check, out, Audio, Book, Elven, Magic, on, Audible, ,, amazon, pay, $, 75, for, every, bounty, ...","[check, out, audio, book, elven, magic, on, audible, ,, amazon, pay, $, 75, for, every, bounty, ...","[check, out, audio, book, elven, magic, on, audible, amazon, pay, 75, for, every, bounty, have, ...","[check, audio, book, elven, magic, audible, amazon, pay, 75, every, bounty, member, 91, days, co...","[(check, NN), (audio, JJ), (book, NN), (elven, RB), (magic, JJ), (audible, JJ), (amazon, NN), (p...","[(check, n), (audio, a), (book, n), (elven, r), (magic, a), (audible, a), (amazon, n), (pay, n),...","[check, audio, book, elven, magic, audible, amazon, pay, 75, every, bounty, member, 91, day, con..."
2,2,1289257452219793408,2020-07-31 17:51:44+00:00,#COVID19KE #TheScoreKE #ruiru,0,2,#COVID19KE #TheScoreKE #ruiru,"[#COVID19KE, #TheScoreKE, #ruiru]",#COVID19KE #TheScoreKE #ruiru,"[#, COVID19KE, #, TheScoreKE, #, ruiru]","[#, covid19ke, #, thescoreke, #, ruiru]","[covid19ke, thescoreke, ruiru]","[covid19ke, thescoreke, ruiru]","[(covid19ke, NN), (thescoreke, NN), (ruiru, NN)]","[(covid19ke, n), (thescoreke, n), (ruiru, n)]","[covid19ke, thescoreke, ruiru]"
3,3,1289191573054345219,2020-07-31 13:29:57+00:00,In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,10,28,#BuildBackBetter #SDGs #COVID19KE,"[In, the, opening, remarks, @Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, us...",In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,"[In, the, opening, remarks, @, Ezra_K_Yego, mentioned, that, it, is, extremely, important, for, ...","[in, the, opening, remarks, @, ezra_k_yego, mentioned, that, it, is, extremely, important, for, ...","[in, the, opening, remarks, ezra_k_yego, mentioned, that, it, is, extremely, important, for, us,...","[opening, remarks, ezra_k_yego, mentioned, extremely, important, us, come, inclusive, collaborat...","[(opening, VBG), (remarks, NNS), (ezra_k_yego, RB), (mentioned, VBD), (extremely, RB), (importan...","[(opening, v), (remarks, n), (ezra_k_yego, r), (mentioned, v), (extremely, r), (important, a), (...","[open, remark, ezra_k_yego, mention, extremely, important, u, come, inclusive, collaborative, am..."
4,4,1289142305593724928,2020-07-31

In [ ]:
#Save to a CSV file for further exploratory data analysis

new_df.to_csv('/content/gdrive/My Drive/Tweets/nairobi_clean.csv')

# <center>**Nairobi;Exploratory Data Analysis**</center>

*The steps below are meant to:*


*   Drive intuition
*   Formulate testable hypotheses
*   Make use of descriptive statistics and visualizations





In [ ]:
#with open('/content/gdrive/My Drive/Tweets/nairobi_pkl', 'rb') as pickle_file:
    #clean_df = pickle.load(pickle_file)

#clean_df = pd.read_pickle('/content/gdrive/My Drive/Tweets/nairobi_pkl', compression=None)
#clean_df = pickle.load(open('/content/gdrive/My Drive/Tweets/nairobi_pkl','rb'))
#Load the clean dataset
clean_df = pd.read_csv('/content/gdrive/My Drive/Tweets/nairobi_clean.csv')
clean_df.head()

,Unnamed: 0,Unnamed: 0.1,ID,DATE,TWEET,RETWEETS,FAVORITES,HASHTAGS,UNCONTRACTED,UNCONTRACTED_STR,TOKENIZED,LOWER,NO_PUNC,STOPWORDS_REMOVED,POS_TAGS,WORDNET_POS,LEMMATIZED
0,0,0,1289314486092877825,2020-07-31 21:38:22+00:00,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,0,1,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy,"['#FACupFinal', '#ruiru', '#lockdownextension', '#COVID19KE', '#TheUmbrellaAcademy', 'https://m....",#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy https://m.facebook.com/stor...,"['#', 'FACupFinal', '#', 'ruiru', '#', 'lockdownextension', '#', 'COVID19KE', '#', 'TheUmbrellaA...","['#', 'facupfinal', '#', 'ruiru', '#', 'lockdownextension', '#', 'covid19ke', '#', 'theumbrellaa...","['facupfinal', 'ruiru', 'lockdownextension', 'covid19ke', 'theumbrellaacademy', 'https', '//m.fa...","['facupfinal', 'ruiru', 'lockdownextension', 'covid19ke', 'theumbrellaacademy', 'https', '//m.fa...","[('facupfinal', 'JJ'), ('ruiru', 'NN'), ('lockdownextension', 'NN'), ('covid19ke', 'NN'), ('theu...","[('facupfinal', 'a'), ('ruiru', 'n'), ('lockdownextension', 'n'), ('covid19ke', 'n'), ('theumbre...","['facupfinal', 'ruiru', 'lockdownextension', 'covid19ke', 'theumbrellaacademy', 'http', '//m.fac..."
1,1,1,1215096387550896128,2020-01-09 02:22:08+00:00,"Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...",28,108,NaN,"['Check', 'out', 'Audio', 'Book', 'Elven', 'Magic', 'on', 'Audible,', 'amazon', 'pay', '$75', 'f...","Check out Audio Book Elven Magic on Audible, amazon pay $75 for every bounty(have to be a member...","['Check', 'out', 'Audio', 'Book', 'Elven', 'Magic', 'on', 'Audible', ',', 'amazon', 'pay', '$', ...","['check', 'out', 'audio', 'book', 'elven', 'magic', 'on', 'audible', ',', 'amazon', 'pay', '$', ...","['check', 'out', 'audio', 'book', 'elven', 'magic', 'on', 'audible', 'amazon', 'pay', '75', 'for...","['check', 'audio', 'book', 'elven', 'magic', 'audible', 'amazon', 'pay', '75', 'every', 'bounty'...","[('check', 'NN'), ('audio', 'JJ'), ('book', 'NN'), ('elven', 'RB'), ('magic', 'JJ'), ('audible',...","[('check', 'n'), ('audio', 'a'), ('book', 'n'), ('elven', 'r'), ('magic', 'a'), ('audible', 'a')...","['check', 'audio', 'book', 'elven', 'magic', 'audible', 'amazon', 'pay', '75', 'every', 'bounty'..."
2,2,2,1289257452219793408,2020-07-31 17:51:44+00:00,#COVID19KE #TheScoreKE #ruiru,0,2,#COVID19KE #TheScoreKE #ruiru,"['#COVID19KE', '#TheScoreKE', '#ruiru']",#COVID19KE #TheScoreKE #ruiru,"['#', 'COVID19KE', '#', 'TheScoreKE', '#', 'ruiru']","['#', 'covid19ke', '#', 'thescoreke', '#', 'ruiru']","['covid19ke', 'thescoreke', 'ruiru']","['covid19ke', 'thescoreke', 'ruiru']","[('covid19ke', 'NN'), ('thescoreke', 'NN'), ('ruiru', 'NN')]","[('covid19ke', 'n'), ('thescoreke', 'n'), ('ruiru', 'n')]","['covid19ke', 'thescoreke', 'ruiru']"
3,3,3,1289191573054345219,2020-07-31 13:29:57+00:00,In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,10,28,#BuildBackBetter #SDGs #COVID19KE,"['In', 'the', 'opening', 'remarks', '@Ezra_K_Yego', 'mentioned', 'that', 'it', 'is', 'extremely'...",In the opening remarks @Ezra_K_Yego mentioned that it is extremely important for us to come up w...,"['In', 'the', 'opening', 'remarks', '@', 'Ezra_K_Yego', 'mentioned', 'that', 'it', 'is', 'extrem...","['in', 'the', 'opening', 'remarks', '@', 'ezra_k_yego', 'mentioned', 'that', 'it', 'is', 'extrem...","['in', 'the', 'opening', 'remarks', 'ezra_k_yego', 'mentioned', 'that', 'it', 'is', 'extremely',...","['opening', 'remarks', 'ezra_k_yego', 'mentioned', 'extremely', 'important', 'us', 'come', 'incl...","[('opening', 'VBG'), ('remarks', 'NNS'), ('ezra_k_yego', 'RB'), ('mentioned', 'VBD'), ('extremel...","[('opening', 'v'), ('remarks', 'n'), ('ezra_k_yego', 'r'), ('mentioned', 'v'), ('extremely', 'r'...","['open', 'remark', 'ezra_k_yego', 'mention

In [ ]:
#Overwrite our dataframe to retain the only columns we are interested in.
'''Re-arrange the columns so as to have the 'Lemmatized' column preceeded by the 'Date' column'''

clean_df = clean_df[['ID','DATE','LEMMATIZED','RETWEETS','FAVORITES','HASHTAGS']]
clean_df.head()

,ID,DATE,LEMMATIZED,RETWEETS,FAVORITES,HASHTAGS
0,1289314486092877825,2020-07-31 21:38:22+00:00,"['facupfinal', 'ruiru', 'lockdownextension', 'covid19ke', 'theumbrellaacademy', 'http', '//m.fac...",0,1,#FACupFinal #ruiru #lockdownextension #COVID19KE #TheUmbrellaAcademy #TheUmbrellaAcademy
1,1215096387550896128,2020-01-09 02:22:08+00:00,"['check', 'audio', 'book', 'elven', 'magic', 'audible', 'amazon', 'pay', '75', 'every', 'bounty'...",28,108,NaN
2,1289257452219793408,2020-07-31 17:51:44+00:00,"['covid19ke', 'thescoreke', 'ruiru']",0,2,#COVID19KE #TheScoreKE #ruiru
3,1289191573054345219,2020-07-31 13:29:57+00:00,"['open', 'remark', 'ezra_k_yego', 'mention', 'extremely', 'important', 'u', 'come', 'inclusive',...",10,28,#BuildBackBetter #SDGs #COVID19KE
4,1289142305593724928,2020-07-31 10:14:11+00:00,"['wish', 'front', 'line', 'worker', 'quick', 'recovery', 'covid19ke']",0,0,#COVID19KE


In [ ]:
#Use TextBlob library to analyze sentiment
#Convert the 'Lemmatized' column to string since textblob only takes in string inputs.

clean_df['LEMMA_STR'] = [' '.join(map(str,l)) for l in clean_df['LEMMATIZED']]
#clean_df.head()
clean_df['SENTIMENT'] = clean_df['LEMMA_STR'].apply(lambda x: TextBlob(x).sentiment.polarity)
clean_df.tail()

,ID,DATE,LEMMATIZED,RETWEETS,FAVORITES,HASHTAGS,LEMMA_STR,SENTIMENT
8636,1247945415967850499,2020-04-08 17:52:27+00:00,['lockdownextension'],0,1,#lockdownextension,[ ' l o c k d o w n e x t e n s i o n ' ],0.0
8637,1247936155166875651,2020-04-08 17:15:39+00:00,"['covid-19', 'pandemic', 'end', 'country', 'music', 'sorry', 'fan', 'leave', '...', 'kenny', 'ro...",0,0,#RealRuto #KunaHopeTutawin #COVID19 #FullMoon #WednesdayMotivation #WednesdayThoughts #lockdowne...,"[ ' c o v i d - 1 9 ' , ' p a n d e m i c ' , ' e n d ' , ' c o u n t r y ' , ' m u s i ...",0.0
8638,1247928683626668033,2020-04-08 16:45:58+00:00,"['know', 'patient', 'south', 'sudan', 'send', 'kenya', 'last', 'one', 'kenya', 'please', 'open',...",0,14,#lockdownextension #SouthSudan #RealRuto,"[ ' k n o w ' , ' p a t i e n t ' , ' s o u t h ' , ' s u d a n ' , ' s e n d ' , ' k ...",0.0
8639,1247916014114877440,2020-04-08 15:55:37+00:00,"['iam_ronoh', 'rate', '10', 'kunahopetutawin', 'covid19', 'wednesdaymotivation', 'stayhome', 'we...",0,1,#KunaHopeTutawin #COVID19 #WednesdayMotivation #StayHome #StayHome #WednesdayThoughts #lockdowne...,"[ ' i a m _ r o n o h ' , ' r a t e ' , ' 1 0 ' , ' k u n a h o p e t u t a w i n ' , ' ...",0.0
8640,1247812906613264385,2020-04-08 09:05:54+00:00,"['wiredea', 'message', 'kenya', 'golf', 'union', 'light', 'announcement', 'c', 'health', 'yester...",0,0,#WiredEA #17mps #coronavirusinKenya #FullMoon #CoronavirusPandemic #StayHome #StayHome #lockdown...,"[ ' w i r e d e a ' , ' m e s s a g e ' , ' k e n y a ' , ' g o l f ' , ' u n i o n ' , ...",0.0


### **Tokenization**

In [ ]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
# Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

spacy_tokenizer(sentence=new_df)

'''#Load model to return language object

nlp = spacy.load('en')

#Calling nlp on our tweet texts to return a processed Doc for each

new_df['doc'] = [nlp(TWEET) for TWEET in new_df.TWEET]

new_df.sample(3)
#new_df.head()'''

NameError: ignored

In [ ]:
#converting HTML entities

print("""First Step : Converting html entities i.e. (&lt; &gt; &amp;)
( "&lt;" is converted to “<” and "&amp;" is converted to “&”)""")

html_parser = HTMLParser()

In [ ]:
# Created a new columns i.e. clean_tweet contains the same tweets but cleaned version

new_df['CLEAN_TWEET'] = new_df['TWEET'].apply(lambda x: html_parser.unescape(x))

new_df.head()

In [ ]:
#Re-arrange the columns so that the 'clean tweets' column is preceeded by the 'tweet' column

'''List of former columns in order:
Unnamed: 0 
ID             
DATE           
TWEET          
RETWEETS       
FAVORITES      
HASHTAGS'''

new_df = new_df[['Unnamed: 0','ID','DATE','TWEET', 'CLEAN_TWEET', 'RETWEETS', 'FAVORITES', 'HASHTAGS']]


#Print out the new order of columns

'''for col in new_df:
  print(col)'''

#view the adjusted dataframe

new_df.head()

In [ ]:
#Removing '@User' from all the tweets

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

# remove twitter handles (@user)
new_df['CLEAN_TWEET'] = np.vectorize(remove_pattern)(new_df['CLEAN_TWEET'], "@[\w]*")

new_df.head(10)

In [ ]:
# Changing all the tweets into lowercase

new_df['CLEAN_TWEET'] = new_df['CLEAN_TWEET'].apply(lambda x: x.lower())
new_df.head(10)

In [ ]:
#Replacing punctuation with space

new_df['CLEAN_TWEET'] = new_df['CLEAN_TWEET'].apply(lambda x: re.sub(r'[^\w\s]',' ',x))

new_df.head(10)

In [ ]:
#Replacing special characters with space

new_df['CLEAN_TWEET'] = new_df['CLEAN_TWEET'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',x))

new_df.head(10)

In [ ]:
#Replacing Numbers(Integers) with space

new_df['CLEAN_TWEET'] = new_df['CLEAN_TWEET'].apply(lambda x: re.sub(r'[^a-zA-Z]',' ',x))

new_df.head(10)

In [ ]:
#Removing words whose length is less than 1

new_df['CLEAN_TWEET'] = new_df['CLEAN_TWEET'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
new_df['CLEAN_TWEET'][0:5]

In [ ]:
#Word Cloud

wordcloud__ = WordCloud(
                          background_color='yellow',
                          stopwords=set(STOPWORDS),
                          max_words=250,
                          max_font_size=40, 
                          random_state=1705
                         ).generate(str(new_df['LEMMATIZED'].dropna()))
def cloud_plot(wordcloud):
    fig = plt.figure(1, figsize=(20,15))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
cloud_plot(wordcloud__)

In [ ]:
#The number of tweets according to dates

df['DATE'] =  pd.to_datetime(df['DATE'])
cnt_srs =df['DATE'].dt.date.value_counts()
cnt_srs = cnt_srs.sort_index()
plt.figure(figsize=(16,12))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color='green')
plt.xticks(rotation='vertical')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of tweets', fontsize=12)
plt.title("Number of tweets according to dates")
plt.show()